# Handin number two: weighted shortest paths

In the [Shortest path](Shortest_paths.ipynb) notes, you saw Dijkstra's and Floyd-Warshall's algorithms for finding the shortest path from one node to all others all pairs, respectively. In those algorithms, we simply counted the number of nodes we would have to go through to move from one node to another. But we can generalised this and put *weights* on edges and instead compute the weight of paths with smallest weights.

We will asume that weights are positive numbers -- the algorithms do not actually generalise to handle negative weights -- and we update the representation of graphs to associate weights with each edge. For adjacency list representations, we can simply represent edges as targets combined with a weight:

In [1]:
def make_list_graph(n_vertices):
    return [[] for _ in range(n_vertices)]
    
def add_list_edge(graph, source, target, weight):
    if target not in graph[source]:
        graph[source].append((weight,target))

In [2]:
g = make_list_graph(6)

add_list_edge(g, 0, 1, 0.1)
add_list_edge(g, 0, 5, 0.4)
add_list_edge(g, 1, 2, 0.5)
add_list_edge(g, 1, 3, 1.4)
add_list_edge(g, 2, 4, 2.3)
add_list_edge(g, 3, 5, 0.4)
add_list_edge(g, 5, 1, 0.1)

print(g)

[[(0.1, 1), (0.4, 5)], [(0.5, 2), (1.4, 3)], [(2.3, 4)], [(0.4, 5)], [], [(0.1, 1)]]


For matrix representations, we can replace the bit matrix with the weight matrix we used in Floyd-Warshall's algorithm, i.e. we represent a missing edge as an infinite distance and existing edges by their numbers.

In [5]:
import numpy as np

def make_matrix_graph(n_vertices):
    weights = np.empty(shape = (n_vertices, n_vertices))
    weights[:,:] = float("Inf")
    return weights

def add_matrix_edge(graph, i, j, weight):
    graph[i,j] = weight

In [7]:
mg = make_matrix_graph(6)

add_matrix_edge(mg, 0, 1, 0.1)
add_matrix_edge(mg, 0, 5, 0.4)
add_matrix_edge(mg, 1, 2, 0.5)
add_matrix_edge(mg, 1, 3, 1.4)
add_matrix_edge(mg, 2, 4, 2.3)
add_matrix_edge(mg, 3, 5, 0.4)
add_matrix_edge(mg, 5, 1, 0.1)

print(mg)

[[ inf  0.1  inf  inf  inf  0.4]
 [ inf  inf  0.5  1.4  inf  inf]
 [ inf  inf  inf  inf  2.3  inf]
 [ inf  inf  inf  inf  inf  0.4]
 [ inf  inf  inf  inf  inf  inf]
 [ inf  0.1  inf  inf  inf  inf]]


Below, I have copied the original versions of the two algorithms. Your task is to update them to work with graphs with weighted edges and compute minimal distances taking the weights into account.

In [8]:
def argmin_seen(dist, seen, processed):
    v, d = None, float("Inf")
    for i, dd in enumerate(dist):
        if seen[i] and not processed[i] and dd < d:
            v, d = i, dd
    return v

def dijkstra(graph, s):
    seen = [False] * len(graph)
    processed = [False] * len(graph)
    dist = [float("Inf")] * len(graph)
    
    dist[s] = 0
    seen[s] = True
    v = s
    while v is not None:
        for w in graph[v]:
            if not processed[w]:
                dist[w] = min(dist[w], dist[v] + 1)
                seen[w] = True
        processed[v] = True
        v = argmin_seen(dist, seen, processed)
    
    return dist

In [9]:
def floyd(graph):
    n = graph.shape[0]
    d = make_dist_matrix(graph)
    new, old = d, d.copy()
    for k in range(n):
        for i in range(n):
            for j in range(n):
                new[i,j] = min(old[i,j], old[i,k] + old[k,j])
        old, new = new, old
    return new